In [1]:
!pip install autokeras

     |████████████████████████████████| 174kB 19.3MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=d4ef1fcc7ba006fe3cd3aaf6f8b2bd6f79b9c40fdd229e849d9915b5e616dade
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=8c775e86af84b748d8a8caaf4139a41a3af7e948997bb8217037dc6099c21525
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
pip install clearml

     |████████████████████████████████| 993kB 23.0MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 


In [3]:
from clearml import Task
task = Task.init(project_name='automl', task_name='automl_gray_model')
from tensorflow import keras


ClearML Task: created new task id=af7aec12664d4ca28c12fa475f1b1c33
ClearML results page: https://demoapp.demo.clear.ml/projects/1be01d2f33074d8d89f15e62b3e4d864/experiments/af7aec12664d4ca28c12fa475f1b1c33/output/log
2021-05-10 16:57:43,455 - clearml.Task - INFO - No repository found, storing script code instead


In [ ]:
# data prep
import pandas as pd
import numpy as np
import os
import sys
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

IMG_WIDTH=128
IMG_HEIGHT=128
img_folder= '/content/drive/My Drive/EECS738/flowers'# change

test_class_name = []
train_class_name = []

test_image_3d_array = np.zeros((IMG_WIDTH,IMG_HEIGHT,4323))

train_image_3d_array = np.zeros((IMG_WIDTH,IMG_HEIGHT,4323))

i = 0
test = 0
train = 0
for dir1 in os.listdir(img_folder):
    for file in os.listdir(os.path.join(img_folder, dir1)):
        if i % 5 == 0:
            image_path = os.path.join(img_folder, dir1, file)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)

            image=np.array(img)
            image = image.astype('float32')
            image /= 255
            test_image_3d_array[:,:,test] = image
            test_class_name.append(dir1)
            i = i + 1
            test += 1
        else:
            image_path = os.path.join(img_folder, dir1, file)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)

            image=np.array(img)
            image = image.astype('float32')
            image /= 255
            train_image_3d_array[:,:,train] = image
            train_class_name.append(dir1)
            i = i + 1
            train += 1


test_image_3d_array = test_image_3d_array[:,:,0:test]
train_image_3d_array = train_image_3d_array[:,:,0:train]

test_target_dict = {k: v for v, k in enumerate(np.unique(test_class_name))}

train_target_dict = {k: v for v, k in enumerate(np.unique(train_class_name))}

test_target_val =  [test_target_dict[test_class_name[i]] for i in range(len(test_class_name))]
train_target_val =  [train_target_dict[train_class_name[i]] for i in range(len(train_class_name))]

test_labels = np.zeros((5,test))
train_labels = np.zeros((5,train))

for i in range(len(test_target_val)):
    test_labels[test_target_val[i],i] = 1

for i in range(len(train_target_val)):
    train_labels[train_target_val[i],i] = 1

print('training dataset size = ',train_image_3d_array.shape)
print('testing dataset size = ',test_image_3d_array.shape)
print('training labels size = ',train_labels.shape)
print('testing labels size = ',test_labels.shape)

print('All executed')


error: ignored

In [4]:
import pandas as pd
import numpy as np
import os
import sys
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

train_data = np.load('/content/drive/My Drive/EECS738/train_data.npy')
train_data = train_data.transpose(2,1,0)
test_data = np.load('/content/drive/My Drive/EECS738/test_data.npy')
test_data = test_data.transpose(2,1,0)
train_label = np.load('/content/drive/My Drive/EECS738/train_label.npy')
train_label = train_label.transpose(1,0)
test_label = np.load('/content/drive/My Drive/EECS738/test_label.npy')
test_label = test_label.transpose(1,0)

In [ ]:
print('training dataset size = ',train_data.shape)
print('testing dataset size = ',test_data.shape)
print('training labels size = ',train_label.shape)
print('testing labels size = ',test_label.shape)

training dataset size =  (3458, 128, 128)
testing dataset size =  (865, 128, 128)
training labels size =  (3458, 5)
testing labels size =  (865, 5)


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
import tensorflow as tf
import autokeras as ak
# input_node = ak.ImageInput()
# output_node = ak.normalization()(input_node)
# output_node1 = ak.ConvBlock()(input_node)
# output_node2 = ak.ResNetBlock(version="v2")(input_node)
# output_node3 = ak.XceptionBlock()(input_node)
# # output_node3 = ak.DenseBlock()(output_node)
# # output_node4 = ak.Embedding()(output_node)

# output_node = ak.Merge()([output_node1, output_node2, output_node3])
# output_node = ak.ClassificationHead()(output_node)

input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=False,
    # Do not do data augmentation.
    augment=True,
)(input_node)
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

tensorboard_callback_train = keras.callbacks.TensorBoard(log_dir='log',histogram_freq=1,
  write_images=True)
# Feed the AutoModel with training data.
auto_model.fit(train_data[:], train_label[:], epochs=50, callbacks=[tensorboard_callback_train])


Trial 1 Complete [00h 05m 04s]
val_loss: 1.8009124994277954

Best val_loss So Far: 1.8009124994277954
Total elapsed time: 00h 05m 04s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
109/109 [==============================] - 20s 143ms/step - loss: 17.8736 - accuracy: 0.7383
Epoch 2/50
109/109 [==============================] - 14s 128ms/step - loss: 16.0697 - accuracy: 0.4821
Epoch 3/50
109/109 [==============================] - 14s 127ms/step - loss: 7.5519 - accuracy: 0.2948
Epoch 4/50
109/109 [==============================] - 14s 126ms/step - loss: 6.9491 - accuracy: 0.4696
Epoch 5/50
109/109 [==============================] - 14s 126ms/step - loss: 5.5209 - accuracy: 0.0443
Epoch 6/50
109/109 [==============================] - 14s 127ms/step - loss: 3.3028 - accuracy: 0.1397
Epoch 7/50
109/109 [==============================] - 14s 126ms/step - loss: 3.2789 - accuracy: 0.5841
Epoch 8/50
109/109 [==============================] - 14s 126ms/step - loss: 5.0620 - accuracy: 0.4224
Ep

In [6]:

# Predict with the best model.
predicted_y = auto_model.predict(test_data)
# Evaluate the best model with testing data.
print(auto_model.evaluate(test_data, test_label))
# get the best performing model
model = auto_model.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_clf.h5')
# save results summary
auto_model.tuner.results_summary()

28/28 [==============================] - 2s 42ms/step - loss: 1.8984 - accuracy: 0.2150
[1.8984334468841553, 0.21502889692783356]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128)]   0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 128, 128)     0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 128, 128, 1)  0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
random_flip (RandomFlip)        (None, 128, 128, 1)  0         